<a href="https://colab.research.google.com/github/sostrikov/-/blob/master/Victor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pandas
!pip install sklearn
!pip install matplotlib

# Репозиторий !!!!
UCI (UCI Machine Learning Repository)
https://archive.ics.uci.edu/ml/datasets.php
http://econ.sciences-po.fr/thierry-mayer/data
https://towardsdatascience.com/top-sources-for-machine-learning-datasets-bb6d0dc3378b

## https://github.com/jupyter/jupyter/wiki/A-gallery-of-interesting-Jupyter-Notebooks#economics-and-finance

https://github.com/rsvp/fecon235/blob/master/nb/fred-eurozone.ipynb

Пример решения задачи множественной регрессии с помощью Python

## Загружаем необходимые библиотеки и файл с данными

In [24]:
import pandas as pd
from pandas import read_csv, DataFrame
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
plt.style.use('ggplot')
#plt.style.use('seaborn-pastel')
print(plt.style.available)
%matplotlib inline


#Загружаем данные из файла на компьютере
#from google.colab import files
#uploaded = files.upload()
#import io
#dataset = pd.read_csv(io.BytesIO(uploaded['ENB2012_data.csv']),decimal=",",delimiter=";")# Dataset is now stored in a Pandas Dataframe

# Загружаем файл из репозитория на GitHub
url = 'https://github.com/sostrikov/juputer/blob/master/ENB2012_data2.csv'

#dataset = pd.read_csv('ENB2012_data.csv',decimal=",",delimiter=";")
# Выводим верхние строки таблицы
print(dataset.head())

print("Колонка Y1:\n{}".format(dataset['Y1']))

['dark_background', 'grayscale', 'seaborn-dark-palette', 'seaborn-white', 'seaborn-ticks', 'seaborn-whitegrid', 'fast', 'seaborn', 'seaborn-poster', 'tableau-colorblind10', 'seaborn-bright', 'seaborn-paper', 'seaborn-dark', 'classic', 'seaborn-notebook', 'Solarize_Light2', 'seaborn-talk', 'seaborn-colorblind', 'fivethirtyeight', 'seaborn-darkgrid', 'seaborn-muted', 'seaborn-pastel', '_classic_test', 'bmh', 'ggplot', 'seaborn-deep']
     X1     X2     X3      X4   X5  X6   X7  X8     Y1     Y2
0  0.98  514.5  294.0  110.25  7.0   2  0.0   0  15.55  21.33
1  0.98  514.5  294.0  110.25  7.0   3  0.0   0  15.55  21.33
2  0.98  514.5  294.0  110.25  7.0   4  0.0   0  15.55  21.33
3  0.98  514.5  294.0  110.25  7.0   5  0.0   0  15.55  21.33
4  0.90  563.5  318.5  122.50  7.0   2  0.0   0  20.84  28.28
Колонка Y1:
0      15.55
1      15.55
2      15.55
3      15.55
4      20.84
5      21.46
6      20.71
7      19.68
8      19.50
9      19.95
10     19.34
11     18.31
12     17.05
13     17.4

## Проверяем на наличие нужных колонок и отсутствие пустых колонок

In [0]:
dataset.columns

## Проверяем корректность типов данных

In [0]:
print(dataset.info())

## Проверяем на наличие нулевых (NULL) данных

In [0]:
dataset.isnull().values.any()

## Определяем размерность данных

In [0]:
dataset.shape

##  Получим некоторую сводную информацию по всей таблице.
Это общее их количество (count), среднее значение (mean), стандартное отклонение (std), минимальное (min), макcимальное (max) значения, медиана (50%) и значения нижнего (25%) и верхнего (75%) квартилей:

In [0]:
dataset.describe()

#dataset.groupby('Y1')['Y2'].describe()

## Определяем корреляцию между ячейками

Построим корреляционную матрицу. Все ее недиагональные положительнын значения относительно Y1 и Y2

In [0]:
dataset.corr()

## Уберем лишние столбцы

### Еще можно удалить строки или столбцы с пустыми значениями:
    удалить столбцы с такими значениями (dataset = dataset.dropna(axis=1)),
    удалить строки с такими значениями (dataset = dataset.dropna(axis=0)).
### Можно заполнить элементы с пустыми значениями
    dataset = dataset.fillna(dataset.median(axis=0), axis=0)


In [0]:

dataset = dataset.drop(['X2','X4','X7'], axis=1)
dataset.head()

## Построим диаграмму рассеивания

In [0]:
dataset.plot.scatter(x='Y1', y='Y2', color = 'green')
plt.title('Зависимость Y2 от Y1')


Функция scatter_matrix из модуля pandas.plotting позволяет построить для каждой количественной переменной гистограмму, а для каждой пары таких переменных – диаграмму рассеяния:

In [0]:
from pandas.plotting import scatter_matrix
scatter_matrix(dataset, alpha=0.05, figsize=(10, 10));

## После обработки данных можно перейти к построению модели.

Для построения модели будем использовать следующие методы:

    Метод наименьших квадратов
    Случайный лес
    Логистическую регрессию
    Метод опорных векторов
    Метод ближайших соседей
Оценку будем производить с помощью коэффициента детерминации (R-квадрат). Данный коэффициент определяется следующим образом:

[LaTeX:R^2 = 1 - \frac{V(y|x)}{V(y)} = 1 - \frac{\sigma^2}{\sigma_y^2}]

, где image — условная дисперсия зависимой величины у по фактору х.
Коэффициент принимает значение на промежутке [LaTeX:[0,1]] и чем он ближе к 1 тем сильнее зависимость.
Ну что же теперь можно перейти непосредственно к построению модели и выбору модели. Давайте поместим все наши модели в один список для удобства дальнейшего анализа:

In [0]:
models = [LinearRegression(), # метод наименьших квадратов
	          RandomForestRegressor(n_estimators=100, max_features ='sqrt'), # случайный лес
	          KNeighborsRegressor(n_neighbors=6), # метод ближайших соседей
	          SVR(kernel='linear'), # метод опорных векторов с линейным ядром
	          LogisticRegression() # логистическая регрессия
	          ]

## Разбиваем данные на обучающую (30%) и тестовую последовательность(70%) используя train_test_split

In [0]:
#separating independent and dependent variable
trg = dataset[['Y1']] # В качестве управляемых переменных Y1,Y2
trn = dataset.drop(['Y1'], axis=1) # В качестве неуправляемых переменных все X

Xtrain, Xtest, Ytrain, Ytest = train_test_split(trn, trg, test_size=0.3, random_state = 11)
N_train = Xtrain.shape 
N_test = Xtest.shape 
print (N_train, N_test)


#Ytrain
#Xtrain


## Устанавливаем разные типы регрессии, обучаем модель, проверяем модель на тестовых данных.

### Линейная регрессия

In [0]:
model1 = LinearRegression()

m1 = str(model1)
model1.fit(Xtrain, Ytrain)

# Predicting the Test set results
Ypred = model1.predict(Xtest)
print("Линейная регрессия - правильность на обучающем наборе: {:.2f}".format(model1.score(Xtrain, Ytrain)))                       
print("Линейная регрессия - правильность на тестовом наборе: {:.2f}".format(model1.score(Xtest, Ytest)))



In [0]:
X2 = dataset.iloc[:, 5].values  # dataset.iloc[:, :-1].values Matrix of independent variables -- remove the last column in this data set
Y2 = dataset.iloc[:, 6].values    # Matrix of dependent variables -- just the last column (1 == 2nd column)



### Случайный лес

In [0]:
model1= RandomForestRegressor(n_estimators=100, max_features ='sqrt')
m1 = str(model1)
model1.fit(Xtrain, Ytrain)

# Predicting the Test set results
#Ypred = model1.predict(Xtest)

print("Случайный лес - правильность на обучающем наборе: {:.2f}".format(model1.score(Xtrain, Ytrain)))                       
print("Случайный лес - правильность на тестовом наборе: {:.2f}".format(model1.score(Xtest, Ytest)))

Мы применили метод score к тестовым данным и тестовым ответам и обнаружили, что наша
модель демонстрирует правильность около 72%. Это означает, что модель
выдает правильные прогнозы для 72% наблюдений тестового набора.

## Метод ближайших соседей

In [0]:
# SVR(kernel='linear'), # метод опорных векторов с линейным ядром
# LogisticRegression() # логистическая регрессия

model1= KNeighborsRegressor(n_neighbors=4)
m1 = str(model1)
model1.fit(Xtrain, Ytrain)

# Predicting the Test set results
Ypred = model1.predict(Xtest)

print("Ближайших соседей - правильность на обучающем наборе: {:.2f}".format(model1.score(Xtrain, Ytrain)))                       
print("Ближайших соседей - правильность на тестовом наборе: {:.2f}".format(model1.score(Xtest, Ytest)))

## Метод логистическая регрессия

In [0]:
# логистическая регрессия
# create and configure model
model1 = LogisticRegression(solver='lbfgs')
 	
# create and configure model
#model1 = LogisticRegression(solver='lbfgs', multi_class='ovr')
m1 = str(model1)
model1.fit(Xtrain, Ytrain)

# Predicting the Test set results
Ypred = model1.predict(Xtest)

print("Метод опорных векторов с линейным ядром - правильность на обучающем наборе: {:.2f}".format(model1.score(Xtrain, Ytrain)))                       
print("Метод опорных векторов с линейным ядром - правильность на тестовом наборе: {:.2f}".format(model1.score(Xtest, Ytest)))

In [0]:
#import matplotlib.pyplot as plt

plt.scatter(Xtrain, Ytrain, color = 'green')
plt.scatter(Xtest, Ytest, color = 'red')   
plt.scatter(Xtest, Ypred, color = 'blue')  # The predicted temperatures of the same X_test input.
plt.plot(Xtrain, model1.predict(Xtrain), color = 'gray')
plt.title('Temperature based on chirp count')
plt.xlabel('Chirps/minute')
plt.ylabel('Temperature')
plt.show()

# Пример на датасете Бостон

In [0]:
from sklearn.datasets import load_boston
boston = load_boston()
r = boston.data.shape
r

In [0]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from distutils.version import LooseVersion
from sklearn.datasets import load_boston
from sklearn.preprocessing import QuantileTransformer, quantile_transform
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import median_absolute_error, r2_score

dataset = load_boston()
target = np.array(dataset.feature_names) == "DIS"

X = dataset.data[:, np.logical_not(target)]
y = dataset.data[:, target].squeeze()
y_trans = quantile_transform(dataset.data[:, target],
                             n_quantiles=300,
                             output_distribution='normal',
                             copy=True).squeeze()



In [0]:
# `normed` is being deprecated in favor of `density` in histograms
if LooseVersion(matplotlib.__version__) >= '2.1':
    density_param = {'density': True}
else:
    density_param = {'normed': True}
    

In [0]:
X, y = make_regression(n_samples=10000, noise=100, random_state=0)
y = np.exp((y + abs(y.min())) / 200)
y_trans = np.log1p(y)

In [0]:
f, (ax0, ax1) = plt.subplots(1, 2)

ax0.hist(y, bins=100, **density_param)
ax0.set_xlim([0, 2000])
ax0.set_ylabel('Probability')
ax0.set_xlabel('Target')
ax0.set_title('Target distribution')

ax1.hist(y_trans, bins=100, **density_param)
ax1.set_ylabel('Probability')
ax1.set_xlabel('Target')
ax1.set_title('Transformed target distribution')

f.suptitle("Synthetic data", y=0.035)
f.tight_layout(rect=[0.05, 0.05, 0.95, 0.95])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Пример на реальных данных Бостон

In [0]:
from sklearn.datasets import load_boston
from sklearn.preprocessing import QuantileTransformer, quantile_transform

dataset = load_boston()
target = np.array(dataset.feature_names) == "DIS"
X = dataset.data[:, np.logical_not(target)]
y = dataset.data[:, target].squeeze()
y_trans = quantile_transform(dataset.data[:, target],
                             n_quantiles=300,
                             output_distribution='normal',
                             copy=True).squeeze()

In [0]:
#Делаем заготовку под две картинки
f, (ax0, ax1) = plt.subplots(1, 2)
# Атрибуты первой картинки
ax0.hist(y, bins=100, **density_param)
ax0.set_ylabel('Probability')
ax0.set_xlabel('Target')
ax0.set_title('Target distribution')

# Атрибуты второй картинки
ax1.hist(y_trans, bins=100, **density_param)
ax1.set_ylabel('Probability')
ax1.set_xlabel('Target')
ax1.set_title('Transformed target distribution')

f.suptitle("Boston housing data: distance to employment centers", y=0.035)
f.tight_layout(rect=[0.05, 0.05, 0.95, 0.95])

# Разбиваем данные на обучающую и тестовую последовательность
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)



In [0]:
#X_train
#y_train

In [0]:
f, (ax0, ax1) = plt.subplots(1, 2, sharey=True)
# Определяем тип регрессии Ridge regression
regr = RidgeCV()
# Обучаем модель
regr.fit(X_train, y_train)

y_pred = regr.predict(X_test)

ax0.scatter(y_test, y_pred)
ax0.plot([0, 10], [0, 10], '--k')
ax0.set_ylabel('Target predicted')
ax0.set_xlabel('True Target')
ax0.set_title('Ridge regression \n without target transformation')
ax0.text(1, 9, r'$R^2$=%.2f, MAE=%.2f' % (
    r2_score(y_test, y_pred), median_absolute_error(y_test, y_pred)))
ax0.set_xlim([0, 10])
ax0.set_ylim([0, 10])

regr_trans = TransformedTargetRegressor(
    regressor=RidgeCV(),
    transformer=QuantileTransformer(n_quantiles=300,
                                    output_distribution='normal'))
# ---------------Обучаем модель----------------------------------------------
regr_trans.fit(X_train, y_train)

y_pred = regr_trans.predict(X_test)

ax1.scatter(y_test, y_pred)
ax1.plot([0, 10], [0, 10], '--k')
ax1.set_ylabel('Target predicted')
ax1.set_xlabel('True Target')
ax1.set_title('Ridge regression \n with target transformation')
ax1.text(1, 9, r'$R^2$=%.2f, MAE=%.2f' % (
    r2_score(y_test, y_pred), median_absolute_error(y_test, y_pred)))
ax1.set_xlim([0, 10])
ax1.set_ylim([0, 10])

f.suptitle("Boston housing data: distance to employment centers", y=0.035)
f.tight_layout(rect=[0.05, 0.05, 0.95, 0.95])

plt.show()

# Пример из книжки (стр. 28). Датасет: Ирисы

In [0]:
from sklearn.datasets import load_iris
iris_dataset = load_iris()

In [0]:
print("Ключи iris_dataset: \n{}".format(iris_dataset.keys()))

In [0]:
print(iris_dataset['DESCR'][:193] + "\n...")

In [0]:
print("Названия ответов: {}".format(iris_dataset['target_names']))

In [0]:
print("Названия признаков: \n{}".format(iris_dataset['feature_names']))

In [0]:
print("Тип массива data: {}".format(type(iris_dataset['data'])))

In [0]:
print("Форма массива data: {}".format(iris_dataset['data'].shape))

In [0]:
print("Первые пять строк массива data:\n{}".format(iris_dataset['data'][:5]))

In [0]:
print("Тип массива target: {}".format(type(iris_dataset['target'])))
print("Форма массива target: {}".format(iris_dataset['target'].shape))


In [0]:
print("Ответы:\n{}".format(iris_dataset['target']))

### Метрики эффективности модели

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
iris_dataset['data'], iris_dataset['target'], random_state=0)

In [0]:
print("форма массива X_train: {}".format(X_train.shape))
print("форма массива y_train: {}".format(y_train.shape))

In [0]:
print("форма массива X_test: {}".format(X_test.shape))
print("форма массива y_test: {}".format(y_test.shape))

In [0]:
# создаем dataframe из данных в массиве X_train
# маркируем столбцы, используя строки в iris_dataset.feature_names
iris_dataframe = pd.DataFrame(X_train, columns=iris_dataset.feature_names)
# создаем матрицу рассеяния из dataframe, цвет точек задаем с помощью y_train
grr = pd.plotting.scatter_matrix(iris_dataframe, c=y_train, figsize=(15, 15), marker='o',
    hist_kwds={'bins': 20}, s=60, alpha=.8)

### Построение модели

In [0]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)


In [0]:
print("Правильность на тестовом наборе: {:.2f}".format(knn.score(X_test, y_test)))

### Получение прогнозов
Теперь мы можем получить прогнозы, применив эту модель к новым
данным, по которым мы еще не знаем правильные метки.

In [0]:
X_new = np.array([[5, 2.9, 1, 0.2]])# Создадим новый набор данных/Обратите внимание, что мы записали измерения по одному цветку в
                                    #двумерный массив NumPy, поскольку scikit-learn работает с
                                    #двумерными массивами данных.
print("форма массива X_new: {}".format(X_new.shape))

In [0]:
# Чтобы сделать прогноз, мы вызываем метод predict объекта knn:
prediction = knn.predict(X_new)
print("Прогноз: {}".format(prediction))
print("Спрогнозированная метка: {}".format(
iris_dataset['target_names'][prediction]))

### Оценка качества модели (стр. 37)
Таким образом, мы можем сделать прогноз для каждого ириса в
тестовом наборе и сравнить его с фактической меткой (уже известным
сортом). Мы можем оценить качество модели, вычислив правильность
(accuracy) – процент цветов, для которых модель правильно
спрогнозировала сорта:

In [0]:
y_pred = knn.predict(X_test)
print("Прогнозы для тестового набора:\n {}".format(y_pred))
print("Правильность на тестовом наборе: {:.2f}".format(np.mean(y_pred == y_test)))